# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.classification import LogisticRegression

In [2]:
spark = SparkSession.builder.appName('logreg_project').getOrCreate()

In [3]:
df = spark.read.csv('../datasets/customer_churn.csv',inferSchema=True,header=True)

In [4]:
df.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [5]:
df.show()

+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|              Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|   Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|
|      Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|
|        Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|
|      Phillip White|42.0|       8010.76|              0| 6.71|     10.0|2014-04-22 12:43:12|13120 Daniel Moun...|           Smith Inc|    1|
|     

In [5]:
df.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [41]:
cols=df.select('Age',
 'Total_Purchase',
 'Years',
 'Num_Sites',
 'Churn')

In [42]:
cols.columns

['Age', 'Total_Purchase', 'Years', 'Num_Sites', 'Churn']

### Encode String values

In [44]:
from pyspark.ml.feature import VectorAssembler

In [45]:
assembler = VectorAssembler(inputCols=[
 'Age',
 'Total_Purchase',
 'Years',
 'Num_Sites'],outputCol='features')

In [46]:
output = assembler.transform(cols)

In [49]:
final_data = output.select('features','Churn')

In [50]:
final_data.show()

+--------------------+-----+
|            features|Churn|
+--------------------+-----+
|[42.0,11066.8,7.2...|    1|
|[41.0,11916.22,6....|    1|
|[38.0,12884.75,6....|    1|
|[42.0,8010.76,6.7...|    1|
|[37.0,9191.58,5.5...|    1|
|[48.0,10356.02,5....|    1|
|[44.0,11331.58,5....|    1|
|[32.0,9885.12,6.9...|    1|
|[43.0,14062.6,5.4...|    1|
|[40.0,8066.94,7.1...|    1|
|[30.0,11575.37,5....|    1|
|[45.0,8771.02,6.6...|    1|
|[45.0,8988.67,4.8...|    1|
|[40.0,8283.32,5.1...|    1|
|[41.0,6569.87,4.3...|    1|
|[38.0,10494.82,6....|    1|
|[45.0,8213.41,7.3...|    1|
|[43.0,11226.88,8....|    1|
|[53.0,5515.09,6.8...|    1|
|[46.0,8046.4,5.69...|    1|
+--------------------+-----+
only showing top 20 rows



### Train Test Split

In [51]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

### Model building

In [52]:
log_reg = LogisticRegression(labelCol='Churn')

In [53]:
model = log_reg.fit(train_data)

In [56]:
model.summary.predictions.describe().show()

+-------+------------------+-------------------+
|summary|             Churn|         prediction|
+-------+------------------+-------------------+
|  count|               638|                638|
|   mean|0.1677115987460815|0.11912225705329153|
| stddev|0.3739031240943129| 0.3241864796221151|
|    min|               0.0|                0.0|
|    max|               1.0|                1.0|
+-------+------------------+-------------------+



### Evaluation

In [57]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [58]:
result = model.evaluate(test_data)
result.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|Churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[27.0,8628.8,5.3,...|    0|[5.17074360026059...|[0.99435174070847...|       0.0|
|[28.0,9090.43,5.7...|    0|[1.09308771966809...|[0.74896271331026...|       0.0|
|[28.0,11204.23,3....|    0|[1.15853010865905...|[0.76106552533737...|       0.0|
|[29.0,13240.01,4....|    0|[6.88623008788974...|[0.99897928439564...|       0.0|
|[29.0,13255.05,4....|    0|[4.16967084162882...|[0.98477794538149...|       0.0|
|[30.0,6744.87,5.1...|    0|[2.66172814466310...|[0.93473017935506...|       0.0|
|[30.0,10183.98,5....|    0|[2.69693808017608...|[0.93684572510490...|       0.0|
|[31.0,8688.21,3.5...|    0|[6.16299969351829...|[0.99789849895407...|       0.0|
|[31.0,10058.87,6....|    0|[4.31169024000561...|[0.98676660835071...|       0.0|
|[31.0,12264.68,

### Using AUC

In [59]:
eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='Churn')

In [60]:
auc = eval.evaluate(result.predictions)
auc

0.7633004141446321

### Predict on brand new unlabeled data

We still need to evaluate the new_customers.csv file!

In [61]:
final_model = log_reg.fit(final_data)

In [63]:
new_data = spark.read.csv('../datasets/new_customers.csv',inferSchema=True,header=True)
new_data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [64]:
# Creating feature column using assembler
test_new_data = assembler.transform(new_data)

In [66]:
# We don't need to select column, because we previously define in log reg that take features col as features.
final_result = final_model.transform(test_new_data)

In [68]:
final_result.select('Company','prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+



Ok! That is it! Now we know that we should assign Acocunt Managers to Cannon-Benson,Barron-Robertson,Sexton-GOlden, and Parks-Robbins!